In [ ]:
!pip install gym pettingzoo

In [ ]:
import numpy as np
from copy import copy
from gymnasium.spaces import Discrete , MultiDiscrete
from pettingzoo import ParallelEnv

### ENVIRONMENT

In [ ]:
class GridWorld(ParallelEnv):
    metadata={

        "name":"Grid World v0"

    }

    def __init__(self):


        self.player1=[1,1]
        self.player2=[1,8]
        self.player3=[8,1]
        self.player4=[8,8]
        #[[1, 1], [1, 8], [8, 1], [8, 8]]
        #[[5, 8], [8, 5], [1, 5], [8, 4]]


        self.target1=[5,8]
        self.target2=[8,4]
        self.target3=[1,5]
        self.target4=[5,1]
        self.walls=[[0,4],[1,4],[2,4],[2,5],[4,0],[4,1],[4,2],[5,2],[5,7],[4,7],[4,8],[4,9],[7,4],[7,5],[8,5],[9,5]]
        self.players_index=[self.player1,self.player2,self.player3,self.player4]
        self.target_index=[self.target1,self.target2,self.target3,self.target4]
        self.actor1_state,self.actor2_state,self.actor3_state,self.actor4_state,self.target_state=self.create_board(self.players_index,self.target_index)
        self.timestep=0
        self.possible_agents=('green','purple','blue','yellow')
        self.rewards={a:0 for a in self.possible_agents}


    def create_board(self, player_index, targets_index):
        """Creates the game board with walls, players, and targets."""
        # Create a 3D array where each layer corresponds to a player's board
        board = np.zeros((4, 10, 10))  # 4 layers for 4 players
        board2 = np.zeros((10, 10))    # Separate board for targets and walls

        # Set walls on all boards
        for (a, b) in self.walls:
            for i in range(4):  # Apply to all player boards
                board[i][a, b] = -1
            board2[a, b] = -1   # Also add walls to the target board

        # Place players on their respective boards
        for i, (a, b) in enumerate(player_index):
            if board[i][a, b] != -1:  # Ensure no overlap with walls
                board[i][a, b] = i+1
            else:
                print(f"Warning: Player {i + 1} position [{a}, {b}] overlaps with an existing element!")

        # Place targets on the target board
        for i, (a, b) in enumerate(targets_index):
            if board2[a, b] != -1:  # Ensure no overlap with walls
                board2[a, b] = (i + 1) * 5
            else:
                print(f"Warning: Target {i + 1} position [{a}, {b}] overlaps with an existing element!")

        # Return each player's board and the target board
        return board[0], board[1], board[2], board[3], board2

    def update_player_position(self, player, action,board):
      row, column = player[0], player[1]

      if action == 0:
          # position remains same
          return [row, column]

      if action == 1 and row > 0:  # up
          new_row = row - 1
          if board[new_row][column] != -1:
              return [new_row, column]


      if action == 2 and column < 9:  # right
          new_column = column + 1
          if board[row][new_column] != -1:
              return [row, new_column]


      if action == 3 and column > 0:  # left
          new_column = column - 1
          if board[row][new_column] != -1:
              return [row, new_column]


      if action == 4 and row < 9:  # down
          new_row = row + 1
          if board[new_row][column] != -1:
              return [new_row, column]

    # Return original position if no valid move
      return [row, column]

    def reset(self, choose:bool):
        self.agents=copy(self.possible_agents)
        self.walls=[[0,4],[1,4],[2,4],[2,5],[4,0],[4,1],[4,2],[5,2],[5,7],[4,7],[4,8],[4,9],[7,4],[7,5],[8,5],[9,5]]
        self.board=np.zeros((10,10))
        self.timestep=0
        self.rewardz=0
        self.player1=[1,1]
        self.player2=[1,8]
        self.player3=[8,1]
        self.player4=[8,8]

        if choose:
            for i in range(4):
                valid_target = False
                while not valid_target:

                    new_target = list(np.random.randint(9, size=2))


                    if new_target not in self.walls:
                        self.target[i] = new_target
                        valid_target = True



        else:
          self.target1=[5,8]
          self.target2=[8,4]
          self.target3=[1,5]
          self.target4=[5,1]
          self.target_index=list((self.target1,self.target2,self.target3,self.target4))
        self.rewards={a:0 for a in self.possible_agents}
        self.players_index=list((self.player1,self.player2,self.player3,self.player4))

        self.actor1_state,self.actor2_state,self.actor3_state,self.actor4_state,self.target_state=self.create_board(self.players_index,self.target_index)
        observations= {'at1':[self.players_index[0],self.target_index[0]],'at2':[self.players_index[1],self.target_index[1]],\
                       'at3':[self.players_index[2],self.target_index[2]],'at4':[self.players_index[3],self.target_index[3]]}#,'board':[self.actor1_state,self.actor2_state,self.actor3_state,self.actor4_state,self.target_state]
        #observations = np.concatenate([np.array(value).flatten() for value in observations.values()])
        infos={a: {} for a in self.agents}
        return observations, infos

    def step(self,actions):
        player1_action=actions[0]#['green']
        player2_action=actions[1]#['purple']
        player3_action=actions[2]#['blue']
        player4_action=actions[3]#['yellow']




        self.player1 = self.update_player_position(self.player1, player1_action,self.actor1_state)
        self.player2 = self.update_player_position(self.player2, player2_action,self.actor2_state)
        self.player3 = self.update_player_position(self.player3, player3_action,self.actor3_state)
        self.player4 = self.update_player_position(self.player4, player4_action,self.actor4_state)


        termi1=(self.player1==self.target1)
        termi2=(self.player2==self.target2)
        termi3=(self.player3==self.target3)
        termi4=(self.player4==self.target4)
        termination=[termi1,termi2,termi3,termi4]

        if not all(termination):
            rewards=self.reward_break(termination,self.rewards)

        else:
            rewards=self.rewards
        self.players_index=list((self.player1,self.player2,self.player3,self.player4))
        self.target_index=list((self.target1,self.target2,self.target3,self.target4))
        self.actor1_state,self.actor2_state,self.actor3_state,self.actor4_state,self.target_state=self.create_board(self.players_index,self.target_index)


        truncation=True if self.timestep==1000 else False
        rewards = {f"agent_{i}": reward for i, reward in enumerate(self.rewards.values())}
        self.timestep+=1
        observations =  {'at1':[self.players_index[0],self.target_index[0]],'at2':[self.players_index[1],self.target_index[1]],\
                         'at3':[self.players_index[2],self.target_index[2]],'at4':[self.players_index[3],self.target_index[3]]}#'board':[self.actor1_state,self.actor2_state,self.actor3_state,self.actor4_state,self.target_state
        #observations = np.concatenate([np.array(value).flatten() for value in observations.values()])
        # Get dummy infos (not used in this example)
        infos = {a: {} for a in self.agents}

        return observations, rewards, all(termination), infos

    def reward_break(self,termins:list,rewards:list):
        rew=rewards.copy()
        for key,term in zip(rew.keys(),termins):
            if not term:
                rewards[key]-=1
        return rewards







### Try 1

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random

# Hyperparameters
NUM_AGENTS = 4
STATE_SIZE = 4  # [player_position, target_position] for each agent
ACTION_SIZE = 5  # [stay, up, right, left, down]
GAMMA = 0.99
LR = 1e-3
BATCH_SIZE = 32
BUFFER_SIZE = int(1e5)
EPSILON_START = 1.0
EPSILON_DECAY = 0.001
EPSILON_MIN = 0.1
UPDATE_EVERY = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size,256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, action_size)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)


In [ ]:
class ReplayBuffer:
    def __init__(self, action_size, buffer_size, batch_size):
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size

    def add(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)
        states, actions, rewards, next_states, dones = zip(*experiences)

        return (torch.tensor(states, dtype=torch.float, device=DEVICE),
                torch.tensor(actions, dtype=torch.long, device=DEVICE),
                torch.tensor(rewards, dtype=torch.float, device=DEVICE),
                torch.tensor(next_states, dtype=torch.float, device=DEVICE),
                torch.tensor(dones, dtype=torch.float, device=DEVICE))

    def __len__(self):
        return len(self.memory)


In [ ]:
class Agent:
    def __init__(self, state_size, action_size, agent_id):
        self.state_size = state_size
        self.action_size = action_size
        self.agent_id = agent_id

        self.qnetwork_local = DQN(state_size, action_size).to(DEVICE)
        self.qnetwork_target = DQN(state_size, action_size).to(DEVICE)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE)
        self.t_step = 0
        self.epsilon = EPSILON_START

    def act(self, state):
        if random.random() > self.epsilon:
            state = torch.tensor(state, dtype=torch.float, device=DEVICE).unsqueeze(0)
            with torch.no_grad():
                action_values = self.qnetwork_local(state)
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def step(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)

        # Learn every UPDATE_EVERY steps
        self.t_step += 1
        if self.t_step % UPDATE_EVERY == 0 and len(self.memory) > BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)

    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences

        Q_expected = self.qnetwork_local(states).gather(1, actions.unsqueeze(1)).squeeze()
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0]
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        loss = nn.MSELoss()(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Update target network
        if self.t_step % 100 == 0:
            self.soft_update(self.qnetwork_local, self.qnetwork_target, tau=1e-3)

    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)



In [ ]:
# Initialize agents
agents = [Agent(STATE_SIZE, ACTION_SIZE, i) for i in range(NUM_AGENTS)]

def train(env, n_episodes=1000, max_t=100):
    scores = []
    for i_episode in range(1, n_episodes + 1):
        obs, _ = env.reset(False)
        states = [np.array(obs[f'at{i+1}']).flatten() for i in range(NUM_AGENTS)]
        scores_episode = np.zeros(NUM_AGENTS)

        for t in range(max_t):
            actions = [agents[i].act(states[i]) for i in range(NUM_AGENTS)]
            next_obs, rewards, done, _ = env.step(actions)

            next_states = [np.array(next_obs[f'at{i+1}']).flatten() for i in range(NUM_AGENTS)]
            for i in range(NUM_AGENTS):
                agents[i].step(states[i], actions[i], rewards[f"agent_{i}"], next_states[i], done)
                scores_episode[i] += rewards[f"agent_{i}"]

            states = next_states
            if done:
                break

        scores.append(np.mean(scores_episode))
        for agent in agents:
            agent.epsilon = max(EPSILON_MIN, agent.epsilon * EPSILON_DECAY)

        if i_episode % 100 == 0:
            print(f"Episode {i_episode}/{n_episodes}, Average Score: {np.mean(scores[-100:]):.2f}")

    #return scores


In [ ]:
env=GridWorld()
train(env) #increase number of episodes

### Try2

In [ ]:
class Q_Learning:
    def __init__(self, epsilon, env):
        self.epsilon = epsilon
        self.env = env
        self.qtable = [self.init_q_table() for _ in range(4)]  # Separate Q-tables for each agent

    def init_q_table(self):
        """Initialize the Q-table for the grid."""
        key = {}
        for row in range(10):
            for column in range(10):
                if [row, column] not in self.env.walls:  # Valid positions
                    key[(row, column)] = np.zeros(5)  # 5 possible actions
                else:
                    key[(row, column)] = np.full(5, -1000)  # Penalize wall positions
        return key

    def epsilon_decay(self, decay=0.005, min_epsilon=0.01):
        """Decay epsilon over time."""
        self.epsilon = max(self.epsilon - decay, min_epsilon)
        return self.epsilon

    def select_action(self, state, agent_index):
        """Select an action using epsilon-greedy policy."""
        if np.random.rand() < self.epsilon:
            return np.random.randint(5)  # Random action
        else:
            return np.argmax(self.qtable[agent_index][tuple(state)])  # Greedy action

    def learn(self, tot_epi, max_steps, gamma,alpha):

        cum_rewards = []

        for epi in range(tot_epi):

            obs, _ = self.env.reset(False)
            states = [obs['at1'][0], obs['at2'][0], obs['at3'][0], obs['at4'][0]]
            total_reward = [0, 0, 0, 0]

            for step in range(max_steps):
                actions = [self.select_action(state, i) for i, state in enumerate(states)]


                new_obs, rewards, terminations, _ = self.env.step(actions)
                new_states = [new_obs['at1'][0], new_obs['at2'][0], new_obs['at3'][0], new_obs['at4'][0]]
                #avg=0
                #for i in range(4):
                #
                #   avg+=rewards*[f"agent_{i}"]*np.max(np.array(list(self.qtable[i][new_states[i]])))

                for i in range(4):
                    old_state = tuple(states[i])
                    new_state = tuple(new_states[i])
                    reward = rewards[f"agent_{i}"]


                    best_future_q = np.max(self.qtable[i][new_state])
                    current_q = self.qtable[i][old_state][actions[i]]
                    self.qtable[i][old_state][actions[i]] +=alpha*(
                        reward + gamma * best_future_q - current_q
                    )

                    # Accumulate rewards
                    total_reward[i] += reward

                states = new_states  # Update states

                # Break loop if all agents are done
                if terminations:
                    break

            # Decay epsilon
            self.epsilon_decay()

            # Track cumulative reward
            cum_rewards.append(total_reward)
            print(f'Reward for agents in episode {epi}:{cum_rewards[-1]}')

        return cum_rewards


In [ ]:


env=GridWorld()
p=Q_Learning(0.9,env)
reward_array=p.learn(100000,100,0.55,0.46)